In [6]:
# Instalar bibliotecas necessárias (se ainda não tiver)
# !pip install wikipedia spacy
 #!python -m spacy download pt_core_news_sm

import wikipedia
import spacy
from collections import Counter

# 1. Buscar texto da Wikipedia sobre Barack Obama
wikipedia.set_lang("pt")
texto_obama = wikipedia.page("Barack Obama").content

# 2. Carregar modelo spaCy em português
nlp = spacy.load("pt_core_news_sm")

# 3. Processar o texto
doc = nlp(texto_obama)

# 4. Tokenização
tokens = [token.text for token in doc]

# 5. POS Tagging (etiquetagem morfossintática)
pos_tags = [(token.text, token.pos_) for token in doc]

# 6. Entidades nomeadas (NER)
entidades = [(ent.text, ent.label_) for ent in doc.ents]

# 7. Frequência de palavras (ignorando pontuação e stopwords)
palavras_validas = [token.text.lower() for token in doc if token.is_alpha and not token.is_stop]
frequencia = Counter(palavras_validas).most_common(10)

# 8. Exibir resultados
print("📄 Texto original (trecho):")
print(texto_obama[:500], "...")

print("\n🔤 Tokens:")
print(tokens[:20])

print("\n🏷️ POS Tags:")
print(pos_tags[:20])

print("\n🧠 Entidades nomeadas:")
for ent in entidades[:10]:
    print(f"{ent[0]} → {ent[1]}")

print("\n📊 Top 10 palavras mais frequentes:")
for palavra, freq in frequencia:
    print(f"{palavra}: {freq}")

📄 Texto original (trecho):
Barack Hussein Obama II (Honolulu, 4 de agosto de 1961) é um advogado e político norte-americano que serviu como o 44.º presidente dos Estados Unidos de 2009 a 2017, sendo o primeiro afro-americano a ocupar o cargo. Nascido em Honolulu, no Havaí, Obama é graduado em ciência política pela Universidade Columbia e em direito pela Universidade de Harvard, onde foi presidente da Harvard Law Review. Também atuou como organizador comunitário, advogado na defesa de direitos civis e ensinou direito const ...

🔤 Tokens:
['Barack', 'Hussein', 'Obama', 'II', '(', 'Honolulu', ',', '4', 'de', 'agosto', 'de', '1961', ')', 'é', 'um', 'advogado', 'e', 'político', 'norte-americano', 'que']

🏷️ POS Tags:
[('Barack', 'PROPN'), ('Hussein', 'PROPN'), ('Obama', 'PROPN'), ('II', 'PROPN'), ('(', 'PUNCT'), ('Honolulu', 'PROPN'), (',', 'PUNCT'), ('4', 'NUM'), ('de', 'ADP'), ('agosto', 'NOUN'), ('de', 'ADP'), ('1961', 'NUM'), (')', 'PUNCT'), ('é', 'AUX'), ('um', 'DET'), ('advogado', 'NO

In [11]:
from transformers import pipeline
import wikipedia
from collections import Counter
import re

# 1. Buscar texto da Wikipedia
wikipedia.set_lang("pt")
texto_obama = wikipedia.page("Barack Obama").content

# 2. Carregar modelo de análise de sentimentos
analise_sentimento = pipeline(
    "sentiment-analysis",
    model="tbluhm/clf-sentimentos-cmts"
)

# 3. Pré-processar texto para frequência de palavras
# Remove pontuação e transforma em minúsculas
palavras = re.findall(r'\b\w+\b', texto_obama.lower())
palavras_filtradas = [p for p in palavras if len(p) > 3]  # Ignora palavras curtas

# 4. Contar frequência
frequencia = Counter(palavras_filtradas).most_common(10)

print("\n📊 Top 10 palavras mais frequentes:")
for palavra, freq in frequencia:
    print(f"{palavra}: {freq}")

# 5. Análise de sentimentos por parágrafo
paragrafos = texto_obama.split("\n")
paragrafos_validos = [p for p in paragrafos if len(p.strip()) > 50]

print("\n🧠 Análise de Sentimentos sobre Barack Obama (Wikipedia):\n")
for i, trecho in enumerate(paragrafos_validos[:5]):
    resultado = analise_sentimento(trecho[:512])[0]
    print(f"Parágrafo {i+1}: {resultado['label']} ({round(resultado['score'], 2)})")
    print(f"Trecho: {trecho[:100]}...\n")

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

Device set to use cpu



📊 Top 10 palavras mais frequentes:
obama: 219
para: 145
como: 66
presidente: 42
mais: 34
governo: 27
durante: 26
após: 25
seus: 25
estados: 24

🧠 Análise de Sentimentos sobre Barack Obama (Wikipedia):

Parágrafo 1: neutral (0.74)
Trecho: Barack Hussein Obama II (Honolulu, 4 de agosto de 1961) é um advogado e político norte-americano que...

Parágrafo 2: neutral (0.66)
Trecho: Em 2004, após vencer a primária democrata da eleição para o Senado em Illinois, Obama foi convidado ...

Parágrafo 3: neutral (0.53)
Trecho: Durante seu primeiro mandato, Obama sancionou propostas de estímulo econômico e outras iniciativas e...

Parágrafo 4: neutral (0.5)
Trecho: Obama foi reeleito presidente em novembro de 2012, derrotando o republicano Mitt Romney, e foi empos...

Parágrafo 5: neutral (0.68)
Trecho: Barack Hussein Obama II nasceu em 4 de agosto de 1961 em Honolulu, Havaí. Sua mãe, Ann Dunham, era b...



In [19]:

from transformers import pipeline
# Carregar o modelo de análise de sentimentos em português
analise_sentimento = pipeline(
    "sentiment-analysis",
    model="tbluhm/clf-sentimentos-cmts"
)

# Função para interpretar o resultado de forma simples
def interpretar_emocao(texto):
    resultado = analise_sentimento(texto)[0]
    sentimento = resultado['label']
    confianca = round(resultado['score'] * 100, 2)

    # Mensagem explicativa para leigos
    if sentimento == "POSITIVE":
        explicacao = "😊 O texto expressa uma emoção positiva — como alegria, aprovação ou entusiasmo."
    elif sentimento == "NEGATIVE":
        explicacao = "😠 O texto expressa uma emoção negativa — como crítica, insatisfação ou tristeza."
    else:
        explicacao = "😐 O texto é neutro — parece informativo ou descritivo, sem emoção clara."

    # Exibir resultado
    print(f"\n🔍 Texto analisado:\n{texto}")
    print(f"\n📊 Resultado: {sentimento} ({confianca}%)")
    print(f"{explicacao}")

# Exemplo de uso
texto_exemplo = "analise_sentimento = pipeline"
interpretar_emocao(texto_exemplo)

Device set to use cpu



🔍 Texto analisado:
analise_sentimento = pipeline

📊 Resultado: neutral (79.81%)
😐 O texto é neutro — parece informativo ou descritivo, sem emoção clara.
